# Testing tunability of models

# todo
- wybrac modele
- wybrac parametry i range parametrow do random searcha
- przeanalizowac wyniki i wykresiki porobic

In [1]:
# Packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [7]:
# diabetes = pd.DataFrame(pd.read_csv("C:/Users/jancw/Desktop/Studia/Sem5/AutoML/Project_1/Datasets/diabetes.csv"))
diabetes = pd.DataFrame(pd.read_csv("./data/diabetes.csv"))
students = pd.DataFrame(pd.read_csv('./data/students.csv'))
# students = pd.DataFrame(pd.read_csv('C:/Users/jancw/Desktop/Studia/Sem5/AutoML/Project_1/Datasets/students.csv'))
students = students.loc[students.Target!='Enrolled']
# telco = pd.DataFrame(pd.read_csv('C:/Users/jancw/Desktop/Studia/Sem5/AutoML/Project_1/Datasets/telco.csv'))
telco = pd.DataFrame(pd.read_csv('./data/telco.csv'))
telco['TotalCharges'] = pd.to_numeric(telco['TotalCharges'],errors='coerce')
telco = telco.iloc[:,1:] # id column
# pistachio = pd.DataFrame(pd.read_csv('C:/Users/jancw/Desktop/Studia/Sem5/AutoML/Project_1/Datasets/pistachio.csv'))
pistachio = pd.DataFrame(pd.read_csv('./data/pistachio.csv'))
datasets = {'diabetes': diabetes,'students': students,'telco': telco,'pistachio': pistachio}

# Wybór danych i rekonesans

In [3]:
# datasets sizes
for key in datasets:
    print(f'{key} len: {len(datasets.get(key))}')

diabetes len: 100000
students len: 3630
telco len: 7043
pistachio len: 2148


In [4]:
# we need about 1500 rows from each set
random_seed = 24
datasets_wanted_size = {}
for key in datasets:
    dataset_size = len(datasets.get(key))
    X = datasets.get(key)
    y = datasets.get(key).iloc[:,-1]
    rest,data_wanted_size = train_test_split(X,test_size=1500/dataset_size,stratify=y,random_state=random_seed)
    datasets_wanted_size[key] = data_wanted_size

In [5]:
# typy kolumn
for key in datasets_wanted_size:
    print(key + ':')
    print(f'{datasets_wanted_size.get(key).dtypes.value_counts()}\n')

diabetes:
int64      4
float64    3
object     2
Name: count, dtype: int64

students:
int64      29
float64     5
object      1
Name: count, dtype: int64

telco:
object     16
int64       2
float64     2
Name: count, dtype: int64

pistachio:
float64    26
int64       2
object      1
Name: count, dtype: int64



In [6]:
# rozkład y
for key in datasets_wanted_size:
    print(key+':')
    print(f'{datasets_wanted_size.get(key).iloc[:,-1].value_counts()}\n')

# zmieniamy dla każdego zbioru ,y na wartości {0,1}
label_encoder = LabelEncoder()
for key in datasets_wanted_size:
    datasets_wanted_size[key].iloc[:,-1] = label_encoder.fit_transform(datasets_wanted_size[key].iloc[:,-1])
    datasets_wanted_size[key] = datasets_wanted_size[key].astype({datasets_wanted_size.get(key).columns[-1]: 'int64'})

diabetes:
diabetes
0    1373
1     127
Name: count, dtype: int64

students:
Target
Graduate    913
Dropout     587
Name: count, dtype: int64

telco:
Churn
No     1102
Yes     398
Name: count, dtype: int64

pistachio:
Class
Kirmizi_Pistachio    860
Siirt_Pistachio      640
Name: count, dtype: int64



Sprawdzanie wartości zmiennyck o typach 'object'

In [7]:
dataset = datasets_wanted_size.get('diabetes')
object_columns = dataset.select_dtypes(include='object')
object_columns_unique_values = {column: dataset[column].unique() for column in object_columns}

for key in object_columns_unique_values:
    print(f'{key}: {object_columns_unique_values.get(key)}')

dataset.head(3)

# nulls :
    # czy w gender: 'Other'==null
    # czy w smoking_history 'No info'==null

# object column encoding :
    # gender - 
    # smoking_history - 



gender: ['Male' 'Female']
smoking_history: ['former' 'never' 'ever' 'No Info' 'not current' 'current']


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
60094,Male,62.0,1,1,former,27.32,4.8,155,0
90264,Male,61.0,0,0,never,58.05,5.7,155,1
53832,Female,73.0,0,0,never,27.32,6.5,200,0


In [8]:
dataset = datasets_wanted_size.get('students')
object_columns = dataset.select_dtypes(include='object')
object_columns_unique_values = {column: dataset[column].unique() for column in object_columns}

for key in object_columns_unique_values:
    print(f'{key}: {object_columns_unique_values.get(key)}')

# kolumny są numeryczne

dataset.head(3)

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
4310,1,15,1,6,1,14,1,23,27,10,...,1,6,12,6,13.625,0,12.4,0.5,1.79,1
4318,1,1,1,2,1,1,14,1,1,10,...,4,11,12,11,15.000,0,8.9,1.4,3.51,1
209,1,12,1,11,1,12,1,3,14,3,...,0,6,6,5,12.800,0,16.2,0.3,-0.92,0


In [9]:
dataset = datasets_wanted_size.get('telco')
object_columns = dataset.select_dtypes(include='object')
object_columns_unique_values = {column: dataset[column].unique() for column in object_columns}

for key in object_columns_unique_values:
    print(f'{key}: {object_columns_unique_values.get(key)}')
    
dataset.head(3)

# object column encoding :
    # wszytkie kolumny wartościach binarnych - label encoding
    # Contract - 
    # TotalCharges - zmiana typu na float, (są wartości ' ' ,które mogą być traktowane jako null)
    # pozostałe - one-hot encoding

gender: ['Female' 'Male']
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
PhoneService: ['No' 'Yes']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['Yes' 'No' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['Yes' 'No' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['Yes' 'No' 'No internet service']
Contract: ['Two year' 'Month-to-month' 'One year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Credit card (automatic)'
 'Bank transfer (automatic)']


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
4726,Female,0,Yes,No,72,No,No phone service,DSL,Yes,Yes,Yes,No,No,Yes,Two year,Yes,Electronic check,49.20,3580.95,0
5892,Male,0,No,No,10,Yes,No,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Mailed check,51.65,524.50,0
274,Male,0,No,No,1,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,No,Mailed check,70.60,70.60,1


In [10]:
dataset = datasets_wanted_size.get('pistachio')
object_columns = dataset.select_dtypes(include='object')
object_columns_unique_values = {column: dataset[column].unique() for column in object_columns}

for key in object_columns_unique_values:
    print(f'{key}: {object_columns_unique_values.get(key)}')

# kolumny są numeryczne

dataset.head(3)

,Area,Perimeter,Major_Axis,Minor_Axis,Eccentricity,Eqdiasq,Solidity,Convex_Area,Extent,Aspect_Ratio,...,StdDev_RR,StdDev_RG,StdDev_RB,Skew_RR,Skew_RG,Skew_RB,Kurtosis_RR,Kurtosis_RG,Kurtosis_RB,Class
1682,80139,1075.2321,418.1198,244.6082,0.8110,319.4310,0.9928,80719,0.7494,1.7093,...,14.5330,15.2768,14.2298,-0.7579,-0.6986,-0.3487,3.4817,3.2926,2.6470,1
89,85969,2345.6279,421.6958,291.1149,0.7235,330.8461,0.8702,98789,0.7033,1.4486,...,18.2206,18.1049,16.7189,-0.9762,-0.9983,-0.7881,3.1975,3.3114,3.3104,0
2021,95334,1251.7111,488.1582,249.2888,0.8598,348.4007,0.9858,96705,0.7721,1.9582,...,23.4085,24.2855,22.0737,-0.6492,-0.4470,-0.4217,2.6227,2.3029,2.2124,1


# Preprocessing

- jeżeli kolumna ma 2 wartości to label encoding, wpp one-hot encoding ,(może coś innego pomyśleć)
- 

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector

In [12]:
# notatnik
# diabetes.nunique()==2

# pip_num = Pipeline(steps=[('impute',SimpleImputer()),
#                           ('scale',MinMaxScaler())])
# pip_cat = Pipeline(steps=[
#     ('ohe',OneHotEncoder()),
#     ('impute',SimpleImputer())])

# col_trans = ColumnTransformer([
#     ("numeric",pip_num,make_column_selector(dtype_include= np.number)),
#     ("cat",pip_cat,make_column_selector(dtype_include= np.object_))
# ])

# model_pipeline = Pipeline([("trans",col_trans),("model",RandomForestClassifier())])
# model_pipeline.fit(datasets_X_test.get('telco'),np.array(datasets_y_test.get('telco')))

# Modele

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# 1 - DecisionTreeClassifier

# class sklearn.tree.DecisionTreeClassifier(*, 
#                                         criterion='gini',
#                                         splitter='best',
#                                         max_depth=None,
#                                         min_samples_split=2, 
#                                         min_samples_leaf=1, 
#                                         min_weight_fraction_leaf=0.0, 
#                                         max_features=None, 
#                                         random_state=None, 
#                                         max_leaf_nodes=None, 
#                                         min_impurity_decrease=0.0, 
#                                         class_weight=None, 
#                                         ccp_alpha=0.0, 
#                                         monotonic_cst=None)


In [ ]:
# 2 - LogisticRegression

# class sklearn.linear_model.LogisticRegression(penalty='l2', 
#                                               *, 
#                                               dual=False, 
#                                               tol=0.0001, 
#                                               C=1.0, 
#                                               fit_intercept=True, 
#                                               intercept_scaling=1, 
#                                               class_weight=None, 
#                                               random_state=None, 
#                                               solver='lbfgs', 
#                                               max_iter=100, 
#                                               multi_class='deprecated', 
#                                               verbose=0, 
#                                               warm_start=False, 
#                                               n_jobs=None, 
#                                               l1_ratio=None)

In [ ]:
# 3 - GradientBoostingClassifier

# class sklearn.ensemble.GradientBoostingClassifier(*, 
#                                                   loss='log_loss', 
#                                                   learning_rate=0.1, 
#                                                   n_estimators=100, 
#                                                   subsample=1.0, 
#                                                   criterion='friedman_mse', 
#                                                   min_samples_split=2, 
#                                                   min_samples_leaf=1, 
#                                                   min_weight_fraction_leaf=0.0, 
#                                                   max_depth=3, 
#                                                   min_impurity_decrease=0.0, 
#                                                   init=None, random_state=None, 
#                                                   max_features=None, 
#                                                   verbose=0, 
#                                                   max_leaf_nodes=None, 
#                                                   warm_start=False, 
#                                                   validation_fraction=0.1, 
#                                                   n_iter_no_change=None, 
#                                                   tol=0.0001, 
#                                                   ccp_alpha=0.0)

In [14]:
# Import the required classifiers from scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Create objects of the classifiers
logistic_regression = LogisticRegression()
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier()
gradient_boosting = GradientBoostingClassifier()
ada_boost = AdaBoostClassifier()
bagging_classifier = BaggingClassifier()
k_neighbors = KNeighborsClassifier()
support_vector_machine = SVC()

# Display the classifier objects
classifiers = {
    "Logistic Regression": logistic_regression,
    "Decision Tree": decision_tree,
    "Random Forest": random_forest,
    "Gradient Boosting": gradient_boosting,
    "AdaBoost": ada_boost,
    "Bagging Classifier": bagging_classifier,
    "K-Neighbors": k_neighbors,
    "Support Vector Machine": support_vector_machine
}

In [15]:
random_seed = 24
datasets_X_train = {}
datasets_X_test = {}
datasets_y_train = {}
datasets_y_test = {}
for key in datasets_wanted_size:
    dataset_size = len(datasets_wanted_size.get(key))
    X = datasets_wanted_size.get(key).iloc[:,:-1]
    y = datasets_wanted_size.get(key).iloc[:,-1]
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=random_seed)
    datasets_X_train[key] = X_train
    datasets_X_test[key] = X_test
    datasets_y_train[key] = y_train
    datasets_y_test[key] = y_test

In [20]:
from timeit import default_timer as timer
tabela_wyników = {}

pip_num = Pipeline(steps=[('impute',SimpleImputer()),
                          ('scale',MinMaxScaler())])
pip_cat = Pipeline(steps=[
    ('ohe',OneHotEncoder()),
    ('impute',SimpleImputer())])
col_trans = ColumnTransformer([
    ("numeric",pip_num,make_column_selector(dtype_include= np.number)),
    ("cat",pip_cat,make_column_selector(dtype_include= np.object_))
])
model_pipeline = Pipeline([("trans",col_trans),("model",RandomForestClassifier())])
model_pipeline.fit(datasets_X_test.get('telco'),np.array(datasets_y_test.get('telco')))

set_key = 'students'

for key in classifiers:
    start = timer()
    model_pipeline = Pipeline([("trans",col_trans),("model",classifiers[key])])
    model_pipeline.fit(datasets_X_train.get(set_key),datasets_y_train.get(set_key))    
    end = timer()
    y_hat = model_pipeline.predict(datasets_X_test.get(set_key))
    tabela_wyników[key] = [accuracy_score(y_hat,datasets_y_test.get(set_key)),end-start] #== [accuracy, time]




# pistachio
# {'Logistic Regression': [0.8766666666666667, 0.014618600020185113],
#  'Decision Tree': [0.8066666666666666, 0.02735049999319017],
#  'Random Forest': [0.8533333333333334, 0.3635247000493109],
#  'Gradient Boosting': [0.8666666666666667, 0.9874855000525713],
#  'AdaBoost': [0.8933333333333333, 0.2399780999403447],
#  'Bagging Classifier': [0.8466666666666667, 0.16180030000396073],
#  'K-Neighbors': [0.8766666666666667, 0.004990700050257146],
#  'Support Vector Machine': [0.89, 0.021320700063370168]}

# diabetes
# {'Logistic Regression': [0.96, 0.02030680002644658],
#  'Decision Tree': [0.94, 0.007484599947929382],
#  'Random Forest': [0.98, 0.11521570000331849],
#  'Gradient Boosting': [0.97, 0.14825059997383505],
#  'AdaBoost': [0.97, 0.078939100028947],
#  'Bagging Classifier': [0.97, 0.02978769992478192],
#  'K-Neighbors': [0.9266666666666666, 0.00779770000372082],
#  'Support Vector Machine': [0.96, 0.011258100043050945]}

# telco
# {'Logistic Regression': [0.8033333333333333, 0.024130700039677322],
#  'Decision Tree': [0.7566666666666667, 0.015992800006642938],
#  'Random Forest': [0.8, 0.15928849996998906],
#  'Gradient Boosting': [0.7966666666666666, 0.24993829999584705],
#  'AdaBoost': [0.8066666666666666, 0.10428309999406338],
#  'Bagging Classifier': [0.7833333333333333, 0.04923330002930015],
#  'K-Neighbors': [0.75, 0.011039600009098649],
#  'Support Vector Machine': [0.81, 0.028400499955751002]}

# students 
# {'Logistic Regression': [0.9066666666666666, 0.011891300091519952],
#  'Decision Tree': [0.8666666666666667, 0.01138309994712472],
#  'Random Forest': [0.9166666666666666, 0.1611720999935642],
#  'Gradient Boosting': [0.9233333333333333, 0.2623509999830276],
#  'AdaBoost': [0.9133333333333333, 0.12784590001683682],
#  'Bagging Classifier': [0.8966666666666666, 0.054303099983371794],
#  'K-Neighbors': [0.8233333333333334, 0.00486270000692457],
#  'Support Vector Machine': [0.8833333333333333, 0.0196890999795869]}


c:\Users\jancw\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [21]:
tabela_wyników

{'Logistic Regression': [0.9066666666666666, 0.016913700033910573],
 'Decision Tree': [0.87, 0.01254030002746731],
 'Random Forest': [0.9266666666666666, 0.15005080006085336],
 'Gradient Boosting': [0.9266666666666666, 0.2406470999121666],
 'AdaBoost': [0.9133333333333333, 0.0902577000670135],
 'Bagging Classifier': [0.9066666666666666, 0.055144599988125265],
 'K-Neighbors': [0.8233333333333334, 0.004798400099389255],
 'Support Vector Machine': [0.8833333333333333, 0.031351299956440926]}